# Naive Bayesian Classifier
### Q1. Bayes Rule을 이해하고 Naive  Bayes classifier가 사용하는 사후 확률 계산 과정을 서술하세요.

- Bayes Rule:   
$P(w_i|x) = \frac{P(x|w_i)|P(w_i)}{P(x)} = \frac{P(x|w_i) P(w_i)}{\Sigma_j P(x|w_j)P(w_j)}$
  -
  - $P(x|w_i)\text{: 사후 확률, posterior}\\
P(x|w_i) \text{: 가능도/우도, likelihood}\\
P(w_i) \text{: 사전 확률, prior}\\
P(x) \text{: 증거, evidence}$

A1.

나이브베이즈 분류모형은 모든 차원의 독립변수가 서로 조건부독립이라는 가정을 사용한다.(naive assumtion)

따라서 나이브베이즈 분류기의 사후 확률 계산 과정은 다음과 같다.

\begin{align*}
P(X | W ) &= \frac{P(W | X) P(X)}{P(W)} \\
&= \frac{P(W = w_1, w_2, \cdots, w_N | X) P(X)}{P(W)} \\
&\text{naive assumption에 의해} \\
&= \frac{\prod_{i=1}^{N} P(W = w_i | X) P(X)}{P(W)}
\end{align*}

최종적으로 분모 𝑃(𝑊)는 모든 클래스 𝑋에 대해 동일하기 때문에, 주어진 데이터에 대한 클래스의 상대적 사후 확률을 계산할 때 생략할 수 있다.



### Q2. Naive Bayes Classification 방법을 이용해서 다음 생성된 리뷰 데이터에 기반한 감정 분석을 해봅시다.

In [26]:
# pip install pandas
import pandas as pd
import re

In [27]:
# 리뷰 데이터 생성
data = {
    'review': [
        'I love this great product! It exceeded my expectations.',
        'The Worst purchase I have ever made. Completely useless.',
        'It is an average product, nothing special but not terrible either.',
        'Great service and who can help but love this design? Highly recommend!',
        'Terrible experience, I will never buy from this poor brand again.',
        'It’s acceptable, but I expected better service, not just an acceptable one.',
        'Absolutely wonderful! I am very satisfied with this great service.',
        'The quality is poor and it broke after one use. Terrible enough!',
        'Acceptable product for the price, but there are better options out there.',
        'Great quality and fast shipping with wonderful service! I love it'
    ],
    'sentiment': [
        'positive', 'negative', 'neutral', 'positive', 'negative',
        'neutral', 'positive', 'negative', 'neutral', 'positive',
    ]
}
df = pd.DataFrame(data)
df.head()

,review,sentiment
0,I love this great product! It exceeded my expe...,positive
1,The Worst purchase I have ever made. Completel...,negative
2,"It is an average product, nothing special but ...",neutral
3,Great service and who can help but love this d...,positive
4,"Terrible experience, I will never buy from thi...",negative


In [28]:
# 불용어 리스트 정의
stopwords = ['i', 'my', 'am', 'this', 'it', 'its', 'an', 'a', 'the', 'is', 'are', 'and', 'product', 'service']

In [29]:
# 텍스트 전처리 함수 정의
def preprocess_text(text):
    # 소문자로 변환
    text = text.lower()
    # 특수 기호 제거
    text = re.sub(r'[^a-z\s]', '', text)
    # 불용어 제거
    words = text.split()
    filtered_words = [word for word in words if word not in stopwords]
    return ' '.join(filtered_words)

# 모든 리뷰에 대해 전처리 수행
df['review'] = df['review'].apply(preprocess_text)

기본적인 데이터 전처리가 완료되었습니다!
이제부터 직접 나이브 베이지안 분류를 수행해 봅시다.  
우리가 분류하고자 하는 문장은 총 두가지 입니다.  
전처리가 완료되었다고 치고,   
첫번째 문장은 **'love, great, awesome'**,  
두번째 문장은 **'terrible, not, never'** 입니다.

사전 확률 $P(positive), P(negative), P(neutral)$을 구합니다.

In [30]:
# 사전 확률 구하는 코드를 작성해주세요.

# 사전 확률 계산 함수 정의
def prior(sentiment):
  num = (df['sentiment'] == sentiment).sum()
  return num/len(df)

# 사전 확률 구하기
print(f'P(positive) = {prior("positive")}')
print(f'P(negative) = {prior("negative")}')
print(f'P(neutral) = {prior("neutral")}')



P(positive) = 0.4
P(negative) = 0.3
P(neutral) = 0.3


가능도를 구하기 위한 확률들을 계산합니다.  
예: 첫번째 문장 분류를 위해서는, $P(love|positive), P(great|positive), P(awesome|positive)\\
P(love|negative), P(great|negative), P(awesome|negative)\\
P(love|neutral), P(great|neutral), P(great|neutral)$를 구합니다.

이 때 CountVectorizer를 사용하여 도출한 단어 벡터를 활용하면 확률들을 간편하게 구할 수 있습니다.  
참고: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

In [31]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
review_array = vectorizer.fit_transform(df['review']).toarray()
review_array

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 2, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 

In [32]:
vectorizer.get_feature_names_out() # 벡터의 각 열이 어떤 단어에 대응하는지

array(['absolutely', 'acceptable', 'after', 'again', 'average', 'better',
       'brand', 'broke', 'but', 'buy', 'can', 'completely', 'design',
       'either', 'enough', 'ever', 'exceeded', 'expectations', 'expected',
       'experience', 'fast', 'for', 'from', 'great', 'have', 'help',
       'highly', 'just', 'love', 'made', 'never', 'not', 'nothing', 'one',
       'options', 'out', 'poor', 'price', 'purchase', 'quality',
       'recommend', 'satisfied', 'shipping', 'special', 'terrible',
       'there', 'use', 'useless', 'very', 'who', 'will', 'with',
       'wonderful', 'worst'], dtype=object)

In [33]:
vectorizer.vocabulary_ # 각 단어에 매핑된 인덱스

{'love': 28,
 'great': 23,
 'exceeded': 16,
 'expectations': 17,
 'worst': 53,
 'purchase': 38,
 'have': 24,
 'ever': 15,
 'made': 29,
 'completely': 11,
 'useless': 47,
 'average': 4,
 'nothing': 32,
 'special': 43,
 'but': 8,
 'not': 31,
 'terrible': 44,
 'either': 13,
 'who': 49,
 'can': 10,
 'help': 25,
 'design': 12,
 'highly': 26,
 'recommend': 40,
 'experience': 19,
 'will': 50,
 'never': 30,
 'buy': 9,
 'from': 22,
 'poor': 36,
 'brand': 6,
 'again': 3,
 'acceptable': 1,
 'expected': 18,
 'better': 5,
 'just': 27,
 'one': 33,
 'absolutely': 0,
 'wonderful': 52,
 'very': 48,
 'satisfied': 41,
 'with': 51,
 'quality': 39,
 'broke': 7,
 'after': 2,
 'use': 46,
 'enough': 14,
 'for': 21,
 'price': 37,
 'there': 45,
 'options': 34,
 'out': 35,
 'fast': 20,
 'shipping': 42}

In [34]:
frequency_matrix = pd.DataFrame(review_array, columns = vectorizer.get_feature_names_out())
frequency_matrix = pd.concat([df['sentiment'], frequency_matrix], axis=1)
frequency_matrix

,sentiment,absolutely,acceptable,after,again,average,better,brand,broke,but,...,terrible,there,use,useless,very,who,will,with,wonderful,worst
0,positive,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,negative,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
2,neutral,0,0,0,0,1,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
3,positive,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,negative,0,0,0,1,0,0,1,0,0,...,1,0,0,0,0,0,1,0,0,0
5,neutral,0,2,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
6,positive,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,1,0
7,negative,0,0,1,0,0,0,0,1,0,...,1,0,1,0,0,0,0,0,0,0
8,neutral,0,1,0,0,0,1,0,0,1,...,0,2,0,0,0,0,0,0,0,0
9,positive,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0


In [35]:
# 위와 같이 조건부 확률을 구하는 코드를 작성해주세요

# 조건부확률 함수
def cp(word, sentiment):
  alpha = 1 # 라플라스 스무딩 계수
  review_count = (prior(sentiment) * len(df)) + alpha # 특정 감정의 리뷰 수 + 라플라스 스무딩 계수
  reviews = frequency_matrix[frequency_matrix['sentiment'] == sentiment] # 특정 감정으로 필터링된 리뷰 df

  word_count_in_sentiments = reviews[word].sum() + alpha if word in reviews.columns else alpha # 필터링된 df에서 특정 단어의 빈도 합산(+ 라플라스 스무딩 계수 추가)

  return word_count_in_sentiments / review_count

print("첫번째 문장")
print(f'P(love | positive) = {cp("love", "positive")}')
print(f'P(great | positive) = {cp("great", "positive")}')
print(f'P(awesome | positive) = {cp("awesome", "positive")}')
print(" ")
print(f'P(love | negative) = {cp("love", "negative")}')
print(f'P(great | negative) = {cp("great", "negative")}')
print(f'P(awesome | negative) = {cp("awesome", "negative")}')
print(" ")
print(f'P(love | neutral) = {cp("love", "neutral")}')
print(f'P(great | neutral) = {cp("great", "neutral")}')
print(f'P(awesome | neutral) = {cp("awesome", "neutral")}')

print(" ")
print("=====================")
print(" ")

print("두번째 문장")
print(f'P(terrible | positive) = {cp("terrible", "positive")}')
print(f'P(not | positive) = {cp("not", "positive")}')
print(f'P(never | positive) = {cp("never", "positive")}')
print(" ")
print(f'P(terrible | negative) = {cp("terrible", "negative")}')
print(f'P(not | negative) = {cp("not", "negative")}')
print(f'P(never | negative) = {cp("never", "negative")}')
print(" ")
print(f'P(terrible | neutral) = {cp("terrible", "neutral")}')
print(f'P(not | neutral) = {cp("not", "neutral")}')
print(f'P(never | neutral) = {cp("never", "neutral")}')

첫 번째 문장
P(love | positive) = 0.8
P(great | positive) = 1.0
P(awesome | positive) = 0.2
 
P(love | negative) = 0.25
P(great | negative) = 0.25
P(awesome | negative) = 0.25
 
P(love | neutral) = 0.25
P(great | neutral) = 0.25
P(awesome | neutral) = 0.25
 
 
두 번째 문장
P(terrible | positive) = 0.2
P(not | positive) = 0.2
P(never | positive) = 0.2
 
P(terrible | negative) = 0.75
P(not | negative) = 0.25
P(never | negative) = 0.5
 
P(terrible | neutral) = 0.5
P(not | neutral) = 0.75
P(never | neutral) = 0.25


독립성 가정을 이용하여 가능도(likelihood)를 구합니다.  
첫번째 문장 예시: $P(love, great, awesome|positive), P(love, great, awesome|negative), P(love, great, awesome|neutral)$

In [38]:
# 가능도 구하는 코드를 작성해주세요.

'''
독립성 가정에 의해 개별 단어의 조건부 확률을 곱해서 전체의 조건부 확률을 계산한다.
'''

# likelihood 계산 함수
def likelihood(words, sentiment):
  likelihood = 1.0
  for word in words:
    likelihood *= cp(word, sentiment)
  return likelihood

# 첫번째 문장 단어
words1 = ["love", "great", "awesome"]

# 첫번째 문장 likelihood
print("첫번째 문장 likelihood")
likelihood1_positive = likelihood(words1, "positive")
likelihood1_negative = likelihood(words1, "negative")
likelihood1_neutral = likelihood(words1, "neutral")

print(f"P(love, great, awesome | positive) = {likelihood1_positive}")
print(f"P(love, great, awesome | negative) = {likelihood1_negative}")
print(f"P(love, great, awesome | neutral) = {likelihood1_neutral}")

print(" ")

# 두번째 문장 단어
words2 = ['terrible', 'not', 'never']

# 두번째 문장 likelihood
print("두번째 문장 likelihood")
likelihood2_positive = likelihood(words2, "positive")
likelihood2_negative = likelihood(words2, "negative")
likelihood2_neutral = likelihood(words2, "neutral")

print(f"P(terrible, not, never | positive) = {likelihood2_positive}")
print(f"P(terrible, not, never | negative) = {likelihood2_negative}")
print(f"P(terrible, not, never | neutral) = {likelihood2_neutral}")

첫번째 문장 likelihood
P(love, great, awesome | positive) = 0.16000000000000003
P(love, great, awesome | negative) = 0.015625
P(love, great, awesome | neutral) = 0.015625
 
두번째 문장 likelihood
P(terrible, not, never | positive) = 0.008000000000000002
P(terrible, not, never | negative) = 0.09375
P(terrible, not, never | neutral) = 0.09375


위에서 구한 사전 확률과 가능도를 이용하여 타겟 문장이 positive, negative, neutral일 확률을 구하고 최종적으로 어떤 감성일지 분석해봅니다.

In [41]:
import numpy as np
# 최종 확률 구하는 코드를 작성해주세요.

# 첫번째 문장 evidence
target1_evidence = (
    likelihood1_positive * prior("positive") +
    likelihood1_negative * prior("negative") +
    likelihood1_neutral * prior("neutral")
)

# 첫번째 문장
# P(positive|target_review1)
pos_pp_1 = (likelihood1_positive * prior("positive")) / target1_evidence
print(f'P(positive|target_review1) = {pos_pp_1}')

# P(negative|target_review1)
neg_pp_1 = (likelihood1_negative * prior("negative")) / target1_evidence
print(f'P(negative|target_review1) = {neg_pp_1}')

# P(neutral|target_review1)
neu_pp_1 = (likelihood1_neutral * prior("neutral")) / target1_evidence
print(f'P(neutral|target_review1) = {neu_pp_1}')

print(" ")

# 두번째 문장 evidence
target2_evidence = (
    likelihood2_positive * prior("positive") +
    likelihood2_negative * prior("negative") +
    likelihood2_neutral * prior("neutral")
)

# 두번째 문장
# P(positive|target_review2)
pos_pp_2 = (likelihood2_positive * prior("positive")) / target2_evidence
print(f'P(positive|target_review2) = {pos_pp_2}')

# P(negative|target_review2)
neg_pp_2 = (likelihood2_negative * prior("negative")) / target2_evidence
print(f'P(negative|target_review2) = {neg_pp_2}')

# P(neutral|target_review2)
neu_pp_2 = (likelihood2_neutral * prior("neutral")) / target2_evidence
print(f'P(neutral|target_review2) = {neu_pp_2}')




P(positive|target_review1) = 0.8722316865417378
P(negative|target_review1) = 0.06388415672913117
P(neutral|target_review1) = 0.06388415672913117
 
P(positive|target_review2) = 0.05382674516400338
P(negative|target_review2) = 0.4730866274179983
P(neutral|target_review2) = 0.4730866274179983


A2-1.   
Target review1의 분류 결과: positive

Target review2의 분류 결과: negative 혹은 neutral

Q2-2. 나이브 베이지안 기반 확률을 구하는 과정에서 어떤 문제점을 발견할 수 있었나요? 그리고 그 문제를 해결하기 위한 방법에 대해 간략하게 조사 및 서술해 주세요. (힌트: Laplace smoothing)

A2-2.

첫번째 문장에서 awesome이라는 단어가 훈련 데이터에서 한 번도 등장하지 않았기 때문에 그에 대한 조건부확률이 0이 되어 계산이 불가능한 문제가 생긴다.

 이 문제를 해결하기 위해 라플라스 스무딩이라는 방법을 사용한다. alpha라는 라플라스 스무딩 계수를 설정하여 초기 빈도값을 0이 아닌 값(ex. alpha = 1)로 설정함으로써 훈련 데이터에서 한 번도 등장하지 않은 단어가 나왔을 때도 확률이 0이 되는 문제를 방지한다.
